In [1]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV
import sktime
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('reconstructed_data.csv') # load the data of EMG signals

df.set_index('sequence',inplace=True) # set the index to the sequence column

X = df.drop('label',axis=1) # drop the label column from the data
Y = df[['label']] # get the label column to be used as the target
#Y = df.drop(df.columns[df.apply(lambda x: x.name != 'label')],axis=1)

In [3]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,14960,14961,14962,14963,14964,14965,14966,14967,14968,14969,14970,14971,14972,14973,14974,14975,14976,14977,14978,14979,14980,14981,14982,14983,14984,14985,14986,14987,14988,14989,14990,14991,14992,14993,14994,14995,14996,14997,14998,14999
sequence,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.100343,0.100350,0.100370,0.100400,0.100435,0.100473,0.100511,0.100550,0.100588,0.100627,0.100666,0.100705,0.100744,0.100783,0.100823,0.100862,0.100902,0.100941,0.100981,0.101021,0.101061,0.101101,0.101141,0.101181,0.101221,0.101261,0.101302,0.101342,0.101383,0.101424,0.101465,0.101506,0.101547,0.101588,0.101629,0.101671,0.101712,0.101754,0.101795,0.101837,...,0.098847,0.098881,0.098915,0.098949,0.098983,0.099018,0.099052,0.099086,0.099121,0.099156,0.099190,0.099225,0.099260,0.099295,0.099330,0.099366,0.099401,0.099437,0.099472,0.099508,0.099544,0.099580,0.099616,0.099652,0.099688,0.099724,0.099761,0.099797,0.099834,0.099871,0.099908,0.099945,0.099982,0.100019,0.100056,0.100093,0.100129,0.100161,0.100185,0.100198
0,0.110339,0.110336,0.110327,0.110314,0.110298,0.110281,0.110264,0.110247,0.110230,0.110213,0.110195,0.110178,0.110161,0.110144,0.110127,0.110110,0.110093,0.110075,0.110058,0.110041,0.110024,0.110007,0.109990,0.109973,0.109956,0.109939,0.109922,0.109905,0.109888,0.109871,0.109854,0.109837,0.109820,0.109803,0.109786,0.109769,0.109752,0.109736,0.109719,0.109702,...,0.111058,0.111040,0.111023,0.111006,0.110989,0.110971,0.110954,0.110937,0.110920,0.110902,0.110885,0.110868,0.110851,0.110833,0.110816,0.110799,0.110782,0.110764,0.110747,0.110730,0.110712,0.110695,0.110678,0.110661,0.110643,0.110626,0.110609,0.110592,0.110574,0.110557,0.110540,0.110523,0.110505,0.110488,0.110471,0.110454,0.110437,0.110423,0.110412,0.110405
0,0.036453,0.036453,0.036452,0.036451,0.036449,0.036447,0.036446,0.036444,0.036442,0.036440,0.036439,0.036437,0.036435,0.036434,0.036432,0.036430,0.036429,0.036427,0.036425,0.036424,0.036422,0.036420,0.036419,0.036417,0.036416,0.036414,0.036412,0.036411,0.036409,0.036408,0.036406,0.036405,0.036403,0.036402,0.036400,0.036399,0.036397,0.036396,0.036394,0.036393,...,0.036537,0.036535,0.036533,0.036531,0.036528,0.036526,0.036524,0.036522,0.036520,0.036518,0.036516,0.036513,0.036511,0.036509,0.036507,0.036505,0.036503,0.036501,0.036499,0.036497,0.036495,0.036493,0.036491,0.036489,0.036487,0.036485,0.036483,0.036481,0.036479,0.036477,0.036475,0.036473,0.036471,0.036469,0.036467,0.036466,0.036464,0.036462,0.036461,0.036460
0,0.020544,0.020545,0.020549,0.020556,0.020563,0.020571,0.020580,0.020588,0.020596,0.020604,0.020613,0.020621,0.020629,0.020637,0.020645,0.020653,0.020662,0.020670,0.020678,0.020686,0.020694,0.020702,0.020710,0.020718,0.020726,0.020734,0.020742,0.020750,0.020758,0.020766,0.020774,0.020782,0.020790,0.020798,0.020806,0.020814,0.020821,0.020829,0.020837,0.020845,...,0.020190,0.020199,0.020207,0.020216,0.020225,0.020233,0.020242,0.020251,0.020259,0.020268,0.020276,0.020285,0.020294,0.020302,0.020311,0.020319,0.020328,0.020336,0.020345,0.020353,0.020362,0.020370,0.020379,0.020387,0.020396,0.020404,0.020413,0.020421,0.020430,0.020438,0.020446,0.020455,0.020463,0.020472,0.020480,0.020488,0.020496,0.020503,0.020509,0.020512
1,0.103840,0.103843,0.103849,0.103859,0.103871,0.103883,0.103896,0.103909,0.103922,0.103935,0.103948,0.103961,0.103974,0.103988,0.104001,0.104014,0.104028,0.104041,0.104054,0.104068,0.104082,0.104095,0.104109,0.104123,0.104136,0.104150,0.104164,0.104178,0.104192,0.104206,0.104220,0.104234,0.104248,0.104263,0.104277,0.104291,0.104306,0.104320,0.104335,0.104349,...,0.103364,0.103374,0.103385,0.103395,0.103406,0.103416,0.103427,0.103437,0.103448,0.103459,0.103470,0.103480,0.103491,0.103502,0.103513,0.103524,0.103536,0.103547,0.103558,0.103569,0.103581,0.103592,0.103604,0.103615,0.103627,0.103638,0.103650,0.103662,0.103674,0.103686,0.103698,0.1037

In [4]:
Y

,label
sequence,
0,0
0,0
0,0
0,0
1,0
...,...
1998,9
1999,9
1999,9


In [5]:
Xarray = X.to_numpy() # convert the data to numpy array
Xarray = Xarray.reshape(2000,4,-1) # reshape the data to 3D numpy array
Xarray

array([[[0.10034339, 0.10035037, 0.10037013, ..., 0.10016064,
         0.10018479, 0.10019794],
        [0.11033939, 0.11033625, 0.11032735, ..., 0.11042262,
         0.11041153, 0.1104055 ],
        [0.03645344, 0.03645311, 0.03645218, ..., 0.03646228,
         0.03646109, 0.03646044],
        [0.02054354, 0.02054505, 0.02054933, ..., 0.02050337,
         0.02050873, 0.02051165]],

       [[0.10384047, 0.10384279, 0.10384935, ..., 0.10378019,
         0.10378812, 0.10379244],
        [0.10463462, 0.10463483, 0.10463543, ..., 0.10462849,
         0.10462936, 0.10462983],
        [0.03283462, 0.03283406, 0.03283249, ..., 0.03285   ,
         0.03284789, 0.03284675],
        [0.01517424, 0.01517493, 0.0151769 , ..., 0.01515572,
         0.0151582 , 0.01515955]],

       [[0.1147981 , 0.11480091, 0.11480885, ..., 0.11472528,
         0.11473484, 0.11474005],
        [0.10165947, 0.10165994, 0.10166129, ..., 0.10164754,
         0.10164906, 0.1016499 ],
        [0.0358137 , 0.03581332, 0.0

In [6]:
Yarray = Y.to_numpy() # convert the target to numpy array
Yarray = Yarray.reshape(2000,4,-1) # reshape the target to 3D numpy array
Yarray = np.mean(Yarray,axis=1).astype(int) # get the mean of the target and convert it to integer to make it a 2D numpy array

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(Xarray, Yarray, test_size = 0.3, random_state=3) # split the data into training and testing set

In [8]:
Y_train.shape, Y_test.shape # check the shape of the target

((1400, 1), (600, 1))

In [9]:
rocket = Rocket(num_kernels=1000,n_jobs=-1) # create a rocket object. set the number of kernels to 1000(default is 10000) and use all available cores

In [10]:
X_train = from_3d_numpy_to_nested(X_train) # convert the training data to nested format
rocket.fit(X_train, Y_train) # fit the rocket object to the training data
X_train = rocket.transform(X_train) # transform the training data

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [11]:
X_test = from_3d_numpy_to_nested(X_test) # convert the testing data to nested format
X_test = rocket.transform(X_test) # transform the testing data

In [12]:
X_train.shape, X_test.shape

((1400, 2000), (600, 2000))

In [13]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)) # create a RidgeClassifier object
classifier.fit(X_train, Y_train) # fit the classifier with the training data

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:1190: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [14]:
Y_pred = classifier.predict(X_test) # predict the target of the testing data
print(classification_report(Y_pred,Y_test)) # print the classification report to check the performance of the model

              precision    recall  f1-score   support

           0       0.94      0.90      0.92        73
           1       0.89      0.86      0.88        58
           2       0.88      0.98      0.93        51
           3       0.72      0.84      0.77        55
           4       0.91      0.73      0.81        84
           5       0.91      0.90      0.91        59
           6       0.78      0.90      0.84        52
           7       0.81      0.86      0.83        63
           8       0.75      0.82      0.78        49
           9       0.85      0.73      0.79        56

    accuracy                           0.85       600
   macro avg       0.85      0.85      0.85       600
weighted avg       0.85      0.85      0.85       600

